In [4]:
import json
from LAC import LAC
from pypinyin import pinyin, lazy_pinyin

with open('text_correct_data/classification/classify_train.json', encoding='utf-8') as f:
    raw_data = json.load(f)

In [10]:
# 装载LAC模型
lac = LAC(mode='lac')
word_cnt = {}

for data in raw_data:
    lac_result = lac.run(data['target'])
    for res in lac_result[0]:
        if not '\u4e00' <= res <= '\u9fa5':
            continue
        if res in word_cnt:
            word_cnt[res] += 1
        else:
            word_cnt[res] = 1

In [46]:
def anteroposterior(word):
    word_py = lazy_pinyin(word)
    candidate = []

    for i, yin in enumerate(word_py):
        req = [yin]
        if yin.endswith('eng'):
            req.append(yin.strip('eng') + 'en')
        elif yin.endswith('en'):
            req.append(yin.strip('en') + 'eng')
        elif yin.endswith('ing'):
            req.append(yin.strip('ing') + 'in')
        elif yin.endswith('in'):
            req.append(yin.strip('in') + 'ing')

        if yin.startswith('zh'):
            req.append(yin.replace('zh', 'z'))
        elif yin.startswith('z'):
            req.append(yin.replace('z', 'zh'))
        elif yin.startswith('ch'):
            req.append('c' + yin.strip('ch'))
        elif yin.startswith('c'):
            req.append('ch' + yin.strip('c'))
        elif yin.startswith('sh'):
            req.append('s' + yin.strip('sh'))
        elif yin.startswith('s'):
            req.append('sh' + yin.strip('s'))

        candidate.append(req)

    requests = []
    for can in candidate:
        if len(can) == 1:
            if not requests:
                requests.append(can[0])
            else:
                for i, req in enumerate(requests):
                    requests[i] = req + '\t' + can[0]
        else:
            if not requests:
                requests = can
            else:
                length = len(requests)
                requests = requests*len(can)
                for i, c in enumerate(can):
                    for j in range(length):
                        requests[i*length+j] = requests[i*length+j] + '\t' + c
    return requests

In [81]:
def check_same_pinyin(word, anteroposterior_flag = True):
    requests = anteroposterior(word) if anteroposterior_flag else ['\t'.join(lazy_pinyin(word))]
    # same_pinyin_dict = same_pinyin_single if len(word) == 1 else same_pinyin_multi
    same_pinyin_dict = same_pinyin
    ret = []
    for req in requests:
        if req in same_pinyin_dict:
            ret += same_pinyin_dict[req]
    if word in ret:
        ret.remove(word)
    return ret

In [69]:
from pypinyin import pinyin, lazy_pinyin

'''
same_pinyin_single = {}
same_pinyin_multi = {}

for word, cnt in word_cnt.items():
    if cnt <= 5:
        continue
    py = '\t'.join(lazy_pinyin(word))
    if len(word) == 1:
        if py in same_pinyin_single:
            same_pinyin_single[py].append(word)
        else:
            same_pinyin_single[py] = [word]
    else:
        if py in same_pinyin_multi:
            same_pinyin_multi[py].append(word)
        else:
            same_pinyin_multi[py] = [word]
'''

same_pinyin = {}
for word, cnt in word_cnt.items():
    if cnt <= 5:
        continue
    py = '\t'.join(lazy_pinyin(word))
    if py in same_pinyin:
        same_pinyin[py].append(word)
    else:
        same_pinyin[py] = [word]

In [92]:
with open('same_pinyin.json', encoding='utf-8') as f:
    same_pinyin = json.load(f)
word = '愤愤'
check_same_pinyin(word, True)

['纷纷', '分分']

In [82]:
check_same_pinyin(word, False)

[]

In [83]:
with open('same_pinyin.json', 'w', encoding='utf-8') as f:
    json.dump(same_pinyin, f, ensure_ascii=False)